<p align="center">
  <img src="https://raw.githubusercontent.com/Lbalkenhol/candl/main/docs/logos/candl_wordmark%26symbol_col_RGB.png" width="750" alt="candl" />
</p>

---

This notebook will introduce you to `candl` (https://arxiv.org/abs/2401.13433). `candl` is a likelihood code that provides easy access to CMB data to allow you to perform more intuitive analysis. Out of the box, `candl` comes with the latest SPT and ACT data installed (primary CMB and lensing power spectrum measurements). Along with the likelihood code, `candl` features an interface module that allows you to easily connect the likelihoods to theory codes (e.g. CAMB, CLASS) and samplers (e.g. Cobaya, MontePython) as well as a tools module that provides short cuts for common calculations. You will explore these in part 1 of this notebook. Moreover, `candl` likelihoods are differentiable, i.e. they allow you to not only obtain the value of the likelihood function, but also its slope. You will learn what this means in detail, how to work with derivatives in practice, and why they are useful in parts 2 and 3 of this notebook.

__Resources:__
* `candl` documentation: https://candl.readthedocs.io/en/latest/
* `candl` GitHub repository: https://github.com/Lbalkenhol/candl
* your expert tutors!

__Overview:__

[Part I: `candl` Basics](#part1)

This part will run you through the basics of using `candl`: how to initialise a likelihood, access different data products, understand the data model, and evaluate the likelihood. For this part of the notebook you will be using the  SPT-3G 2018 TT/TE/EE data set.

[Part II: Building a Differentiable Pipeline, Calculating Derivatives](#part2)

In this part you will build a differentiable pipeline from cosmological parameters all the way to the likelihood value. We will look at two useful applications using the ACT DR4 TT/TE/EE data set.

[Part III: Gradient-Powered Likelihood Exploration](#part3)

In this part you will find the best-fit point of the ACT DR4 TT/TE/EE data set using traditional and gradient-powered methods.

__Disambiguation__:

The word "likelihood" gets thrown around a lot. Depending on the context it can mean different things, such as:

* A piece of software: `candl` is a likelihood code.
* A mathematical function: the function $\mathcal{L}(\theta)$, where $\theta$ are some model parameters.
  * Related: a concept in Bayesian statistics.
* A python function: the implementation of the mathematical function.


## Requirements

This notebook has a few requirements for packages and codes, beyond what you have installed so far. Execute the cell below, it will automatically install the required packages for you and download an archive with additional data into the directory where you have placed this notebook.

The second cell then features our required python imports and some setup for plotting.

In [ ]:
# Install required packages
# Assumes you have previously cloned and installed candl from GitHub
!pip install jax==0.4.24
!pip install jaxlib==0.4.24
!pip install cosmopower-jax
!pip install camb

# Grab the CosmoPower emulator models
!unzip cosmopower_models.zip
!rm cosmopower_models.zip
!rm -r __MACOSX

In [ ]:
# Prepare plotting
%matplotlib inline

# Standard imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

# Import candl
import candl
import candl.data

# Make plots pretty
import candl.plots
candl.plots.set_plot_style()


---

<a id='part1'></a>

# PART I: `candl` Basics

Let's learn the basics of `candl`, how to access different parts of the data, evaluate the likelihood, and use transformations.

<span style="color:blue">
<b>Background:</b>
</span>

Generally, CMB likelihoods can be written as:

$$ -2\log{\mathcal{L}(\theta)} = \left\{\hat{D} - \mathcal{T}\left[D^{\mathrm{CMB}}(\theta), \theta \right]\right\}^T \mathcal{C}^{-1} \left\{\hat{D} - \mathcal{T}\left[D^{\mathrm{CMB}}(\theta), \theta \right]\right\}, $$

where $\mathcal{L}$ is the likelihood, $\theta$ are the (cosmological and nuisance) parameters, $D^{\mathrm{CMB}}$ is the CMB power spectrum, $\mathcal{C}$ the band power covariance matrix and $\mathcal{T}$ the data model. The data model represents a series of transformations that need to be applied to the CMB power spectrum so that it can be compared to the data; this includes accounting for e.g. foreground contamination or calibration uncertainty.

__`candl`__ likelihoods take a dictionary populated with (cosmological and nuisance) parameters, as well as CMB spectra as an input and return the negative log-likelihood value. The data model is implemented as a series of transformations that are applied to the theory spectra in a specific order. These transformations are specified in a `.yaml` file, along with all other necessary information (such as information where to find the band powers, what kind of spectra are being supplied, etc.).

We'll start by initialising a candl likelihood for the SPT-3G 2018 TT/TE/EE data set. Programatically, `candl` likelihoods are python classes and their `log_like` function implements the equation above. Execute the cell below and inspect the output produced by the initialisation paying attention to the information on spectra and the data model.


In [ ]:
# Initialise the SPT-3G 2018 TT/TE/EE likelihood
candl_like = candl.Like(candl.data.SPT3G_2018_TTTEEE)

<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Data Access

`candl` likelihoods have a lot of useful attributes that allow us to directly access different parts of the data set. Let's get an idea for the information that is available. 

<span style="color:red">
<b>Exercise:</b>
</span>

Plot the band powers of the data set, including error bars. You can decide for yourself whether it is best to show all data together, or whether it is easier to look at a given spectrum individually.

<span style="color:green">
<b>Tips:</b>
</span>

The band powers in `candl` are stored as a long vector with all measurements concatenated. The order is specified by the atrtibute `.spec_order` (which matches the output of the initialisation above). The following attributes will also help you:

* `.spec_order`: a list specifying the order of the spectra in the data vector
* `.data_bandpowers`: a long vector of band powers
* `.covariance`: the covariance matrix
* `.bins_start_ix`, `.bins_stop_ix`: lists that contain start and stop indices for the spectra
* `.N_bins`: a list containing the number of bins of each spectrum
* `.effective_ells`: the weighted bins centres of all all spectra

<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Data Structure and Data Model

Let's peel back the curtain a bit. As mentioned before, `candl` likelihoods require `.yaml` files with some specific information inside them in order to be initialised. These `.yaml` files follow the following structure:

<pre style="margin-left: 25pt">
name: [name of data set]
band_power_file: [relative path of the band power file]
covariance_file: [relative path of the band power file]
window_functions_folder: [relative path of the window functions folder]

spectra_info:
   - [spectrum id]: [number of bins]

data_model:
   - [transformation]

priors:
   - [par_names]: [name of parameter to apply the prior to]
     [central_value]: [central value of the prior]
     [prior_std]: [width of the prior (standard deviation)]
</pre>

Let's zoom in on the data model. Execute the cell below and look at the list of transformations modules.

In [ ]:
candl_like.data_model

<span style="color:blue">
<b>Background:</b>
</span>

We can access directly the transformations that make up the data model of the likelihood. Each transformation has an `.output()` function, which takes as the input a dictionary of parameters it requires to compute and returns a long vector of the changes to be applied. The returned vector contains the contributions for all spectra matching the order of the data vector, though it is unbinned, i.e. of length `candl_like.N_spectra_total x len(candl_like.ells)`. Most transformations (such as foregrounds) are additive, apart from calibration, which is multiplicative. 

<span style="color:red">
<b>Exercise:</b>
</span>

Focus on the CIB clustering mdule. Inspect the `.yaml` file of the data set (you can see its location on your machine via `candl_like.data_set_file`) and identify the part that specifies the CIB clustering module. Find the names of the parameters required to evaluate it. Check for their default values in the prior block of the `.yaml` file and obtain predictions from the module for these paramter values. Plot the (non-zero) predictions for all spectra to compare their amplitudes.

Then, sample from the prior and plot different draws of the foreground contribution for one spectrum of your choice (e.g. TT 220x220). Show the chosen band powers with error bars in the same panel.

<span style="color:green">
<b>Tips:</b>
</span>

On google colab, click on the folder icon on the left to open the file browser. If you have trouble locating the file, you can also view it on [GitHub](https://github.com/Lbalkenhol/candl/blob/main/candl/data/SPT3G_2018_TTTEEE_v0/SPT3G_2018_TTTEEE.yaml).


<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Evaluating the Likelihood, Calculating Residuals

Having familiarised ourselves with different aspects of the data and how the code functions, it's about time we obtain a likelihood value! As mentioned above, the `log_like` (and also the `chi_square`) functions of `candl` likelihoods take as input a dictionary with CMB spectra and (cosmological and nuisance) parameter values. We can obtain the CMB spectra from a Boltzmann solver; here, we will use `CAMB`, which you may already be familiar with from other sessions. If not, `CAMB` returns CMB spectra for a given combination of cosmological parameters. The cells below show you an example of how to interface `candl` with `CAMB`.

In [ ]:
import camb
import candl.interface

In [ ]:
# Set up CAMB with some baseline accuracy settings
CAMB_pars = camb.CAMBparams()
CAMB_pars.set_for_lmax(candl_like.ell_max+200, lens_potential_accuracy=2)

# This creates a function that takes a dictionary of cosmological parameters as an input and returns a dictionary of CMB spectra
CAMB_pars_to_theory_specs = candl.interface.get_CAMB_pars_to_theory_specs_func(CAMB_pars)

# Define fiducial parameters, cosmology matches Planck 2018 best fit
fid_pars = {'H0': 67.37,
            'ombh2': 0.02233,
            'omch2': 0.1198,
            'logA': 3.043,
            'ns': 0.9652,
            'tau': 0.054}

# Nuisance parameters are set to the central values of their priors
for par_name in candl_like.required_nuisance_parameters:
    for prior in candl_like.priors:
        if par_name in prior.par_names:
            fid_pars[par_name] = prior.central_value[prior.par_names.index(par_name)]


In [ ]:
# Get the theory spectra from CAMB
CAMB_CMB_Dls = CAMB_pars_to_theory_specs(fid_pars, candl_like.ell_max)

# Put things into the right format for candl
pars_for_like = deepcopy(fid_pars)
pars_for_like["Dl"] = CAMB_CMB_Dls

# Hand off to the likelihood
logl_value = candl_like.log_like(pars_for_like)
chisq_value = candl_like.chi_square(pars_for_like)

# Print results
# Note that for likelihoods with a varying beam covariance matrix (such as SPT-3G 2018 TT/TE/EE) the chi-square value is not twice the log-likelihood value,
# plus the logl function also applies any priors
print(f"logL = {float(logl_value)}")
print(f"chisq = {float(chisq_value)}")

Evaluating a likelihood is great, but often we want to get a deeper understanding of why the data likes or dislikes a certain model. For this purpose, it can be handy to calculate the difference between the data and the model and search for features in these residuals. The `candl.tools.pars_to_model_specs()` function is helpful here: it returns the binned and unbinned model spectra. Under the hood, it queries the `CAMB_pars_to_theory_specs` function, then applies all transformations, and finally, bins the spectra.

In [ ]:
import candl.tools

In [ ]:
# Obtain the model spectrum
model_spectrum, binned_model_spectrum = candl.tools.pars_to_model_specs(candl_like, fid_pars, CAMB_pars_to_theory_specs)

# Calculate error bars
# Note that we need to add the beam correlation matrix to the covariance - a peculiarity of the SPT-3G 2018 TT/TE/EE data set
error_bars = np.sqrt(np.diag(candl_like.covariance) + np.diag(candl_like.beam_correlation) * binned_model_spectrum**2)

# Calculate residuals: model - data
residuals = binned_model_spectrum - candl_like.data_bandpowers
relative_residuals = residuals / error_bars


In [ ]:
# Plot the residuals
plt.close()

# Set figure size
fig = plt.gcf()
fig.set_size_inches(3 * 3.464, 1 * 3.464)

# Plot residuals and horizonal line
plt.plot(relative_residuals)
plt.axhline(0, color="k", lw=0.5)

# Dividers for spectra
for i, spec in enumerate(candl_like.spec_order[:-1]):    
    plt.axvline(candl_like.bins_stop_ix[i], color="k", ls="--", lw=0.5)

# Finish plot with limits, ticks, labels
plt.xlim(0, len(relative_residuals)-1)
plt.ylim((-8, 8))
plt.xticks(candl_like.bins_start_ix+(candl_like.bins_stop_ix-candl_like.bins_start_ix)/2,
           candl_like.spec_order,
           rotation=45)
plt.subplots_adjust(bottom=0.2)
plt.ylabel(r"$\Delta D_\ell / \sigma$")

plt.show()


<span style="color:red">
<b>Exercise:</b>
</span>

Calculate residuals for two more sets of model spectra by varying your favourite cosmological parameter. Increase and Decrease it from the fiducial value and plot the residuals for all three model on the same panel. Do you have an intuition for the shape the residuals take given the parameter you varied?

<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Discussion

In this part, you have learned the basics of `candl` and carried out key parts of high-level CMB data analysis. Making (and interpreting) residual plots is a key part of analysing CMB data - you'll be hard-pressed to find a CMB power spectrum paper without such a plot. With the data band powers at your fingertips and the ability to access the data model directly, you have great tools at your disposal to develop a deeper understanding of the model constraints produced by a given data set.

The `candl.tools` module features a lot more functionality that goes beyond what we've accessed in this part. For example `candl.tools.get_foreground_contributions()` can return the contributions of all foreground modules in different formats, `candl.tools.undo_transformations` can be used to obtain a CMB-only (foreground-free) version of the data band powers, and you can use `candl.tools.generate_mock_data` to generate mock band powers.

---

<a id='part2'></a>

# PART II: Building a Differentiable Pipeline, Calculating Derivatives

`candl` can not only be used to evaluate $\mathcal{L}(\theta)$, but also $\partial\mathcal{L}/\partial \theta$ thanks to `JAX` and the magic of automatic differentiation. We will now compute derivatives of the likelihood and use them in two example calculations.  In doing so, we will also learn how to use the neural network-based emulators of the CMB power spectrum `CosmoPower`.

<span style="color:blue">
<b>Background:</b>
</span>

__`JAX`__ (https://jax.readthedocs.io/en/latest/index.html) is a google-developed python library. For the purpose of this school you can think of it as a smart numpy replacement. `candl` optionally uses `JAX`, meaning that everything you have done up until now did not require `JAX`. However, if `JAX` is installed, `candl` likelihoods run faster and become differentiable through "auto-diff".

__Automatic differentiation__, or "auto-diff", is a computer science concept that is seeing more and more applications in scientific computing and in particular cosmology (see e.g. https://arxiv.org/abs/2302.05163). It is a way to obtain derivatives of functions you write with respect to their input parameters - without resorting to finite difference methods. How auto-diff works is beyond the scope of this school, though if you are interested take a look at https://jax.readthedocs.io/en/latest/automatic-differentiation.html.

__`CosmoPower`__ (https://arxiv.org/abs/2106.03846) is a framework for neural network-based emulators of the CMB power spectrum. You can think of it as a short-cut for evaluating Boltzmann solvers, such as CAMB. `CosmoPower` models are trained on a series of pre-calculated CMB spectra and perform effectively an interpolation between the known points in order to avoid slow CAMB evalutations. Not only do they provide a speed-up, but `CosmoPower` models are also differentiable. Below, we will use `CosmoPower-JAX` (https://arxiv.org/abs/2305.06347) to stay within the same code eco-system and use models trained on high-precision CAMB spectra for the SPT-3G 2018 TT/TE/EE analysis (https://arxiv.org/abs/2212.05642, https://github.com/alessiospuriomancini/cosmopower/tree/main/cosmopower/trained_models/SPT_high_accuracy).

The diagram belows how the different codes work together to create a pipeline that is differentiable. We start out on the left with our input parameters: some cosmological (top blue box) and some nuisance parameters (bottom blue box). The cosmological parameters are fed into `CosmoPower`, which produces a CMB power spectrum ($D_\ell^{CMB}$). This power spectrum in return is then passed to `candl` along with any nuisance parameters and `candl` calculates the likelihood value, $\mathcal{L}$. Since all codes in the black brackets are differentiable, we can also calculate the derivate of the likelihood with respect to any of the input parameters (cosmological and nuisance), $\partial\mathcal{L}/\partial \theta$.

<p align="center">
  <img src="https://raw.githubusercontent.com/Lbalkenhol/candl/main/notebooks/SPT_ACT_summer_school_2024/diff_pipeline.png" width="80%" alt="Differentiable Pipeline"/>
</p>

Why do we bother with this? There are many useful calculations in cosmology that use derivatives of the observables with respect to cosmological parameters. Having access to reliable derivatives, without resorting to finite difference calculations, which can be plagued by numerical issues, trivialises a lot of these calculations. In this part, we will work through some examples that will give you a taste for how gradient information can be useful.

To get started, we will switch data sets to the ACT DR4 TT/TE/EE data. We start by loading it into `candl`. Inspect the output that is produces for a moment. It only features two sets of band powers for each spectrum, with `d` and `w` denoting observations from the ACT deep and wide fields, respectively. This likelihood is already marginalised over foregrounds, providing us with CMB-only band powers. The only nuisance parameter remaining is `yp`, a final calibration of the data. Then we load our CosmoPower models.


In [ ]:
# Initialise the ACT DR4 TT/TE/EE likelihood
candl_like = candl.Like(candl.data.ACT_DR4_TTTEEE)

# Define our fiducial parameters again - now without the SPT nuisance parameters, but adding ACT calibration
fid_pars = {'H0': 67.37,
            'ombh2': 0.02233,
            'omch2': 0.1198,
            'ns': 0.9652,
            'logA': 3.043,
            'tau': 0.054,
            'yp': 1.0}

In [ ]:
# Grab a theory calculator and initialise it
# Here, we use our differentiable, high-precision CosmoPower models
# These take care of the step of moving from cosmological parameters to theory CMB spectra
# There may be some warnings printed, but unless there are any errors you are all good
cp_emulator_filenames = {"TT": "./cosmopower_models/cmb_spt_TT_NN.npz",
                         "TE": "./cosmopower_models/cmb_spt_TE_PCAplusNN.npz",
                         "EE": "./cosmopower_models/cmb_spt_EE_NN.npz"}
CP_pars_to_theory_specs = candl.interface.get_CosmoPowerJAX_pars_to_theory_specs_func(cp_emulator_filenames)


<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Fisher Matrices and Forecasting

<span style="color:blue">
<b>Background:</b>
</span>

Forecasting parameter constraints given data error bars is a common task in cosmology. The most straightforward approach is to use the Fisher information matrix, which, when inverted, gives the expected parameter covariance matrix.

For CMB experiments, we can calculate the Fisher matrix as follows:

$$ F = \frac{\partial D_\ell}{\partial \theta}^T C^{-1} \frac{\partial D_\ell}{\partial \theta}, $$

where $F$ is the Fisher matrix, $\partial D_\ell / \partial \theta$ the derivative of the model spectra with respect to parameters, and $C$ is the band power covariance matrix. This encapsulates the constraining power of the data and the data only, i.e. it does not take into account the effect of priors. These need to be included manually, by adding the inverse covariance to the relevant parameters [1]. For the ACT data, the only parameter with a prior is the optical depth to reionisation, $\tau$.

<span style="color:red">
<b>Exercise:</b>
</span>

Use Fisher matrices to study the impact of the prior on $\tau$, the optical depth to reionisation, on constraints from the ACT DR4 data in $\Lambda \mathrm{CDM}$. Compare with the default choice (a Gaussian prior of width $0.015$ (standard deviation)) to constraints when replacing the $\tau$ prior on the ACT data with the choice of the SPT-3G 2018 TT/TE/EE analysis (standard deviation of $0.0074$). Does the difference in the choice of prior width matter? Which parameters are most effected?

<span style="color:green">
<b>Tips:</b>
</span>

From `jax.jacrev(func)` you obtain a function that efficiently evaluates the derivative of `func`. For more information see: https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html

[1] If you are wondering why, consider that the contribution from a prior on parameter $\tau$ to the log-likelihood is $0.5*[(\tau-\tau_0)/\sigma_{\tau}]^2$, where $\tau_0$ is the central value of the prior and $\sigma_{\tau}$ its width. The Fisher matrix holds the second derivatives of the log-likelihood with respect to the parameters; taking the second derivative of the expression above, only $1/\sigma_{\tau}^2$ survives.


In [ ]:
import jax

In [ ]:
# This plotting routine can help you visualise the parameter covariance matrix
# Look at the doc string below to understand how to use it
def plot_confidence_ellipse(par1, par2, par_cov, par_order, colour):
    """
    Adds 68% and 95% confidence ellipses to the plot for two parameters.
    
    par1: str
        The name of the parameter along the x-axis
    par2: str
        The name of the parameter along the y-axis
    par_cov: array
        The parameters covariance matrix
    par_order: list of strings
        The order of parameters in the covariance matrix
    colour:
        The colour of the contours - any valid matplotlib specification should work
    """
    ax = plt.gca()
    for i in [2,1]:
        candl.plots.add_confidence_ellipse(ax,
                                           par_cov,
                                           par_order.index(par1),
                                           par_order.index(par2),
                                           fid_pars[par1],
                                           fid_pars[par2],
                                           i,
                                           ec=colour,
                                           facecolor="None")

<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## ACT DR4 TE Artificial Recalibration

<img src="https://raw.githubusercontent.com/Lbalkenhol/candl/main/notebooks/SPT_ACT_summer_school_2024/aiola20_fig14b.png" width="400" align="left" alt="Aiola et al. 2020 Figure 14"/>

<span style="color:blue">
<b>Background:</b>
</span>

The image on the left shows constraints placed by the ACT DR4 TT/TE/EE data in $\Lambda \mathrm{CDM}$ in the $n_s - \Omega_b h^2$ plane (blue contours) (taken from https://arxiv.org/abs/2007.07288). The authors note a preference for lower $\Omega_b h^2$ and higher $n_s$ data compared to Planck and WMAP results (purple and yellow/green contours, respectively).

The size of the difference of the ACT results w.r.t. Planck is compatible with a statistical fluctuation. Furthermore, the authors see no evidence for such a re-calibration of their $TE$ power spectrum in their data. Nevertheless, they show how their results shift when artificially multiplying their $TE$ band powers by 5% (light and dark blue contours) and can in such a way be brought to a closer agreement with the satellite data.

In order to produce these results, new MCMC runs with the TE power spectrum artifically multiplied by 0.95 and 1.05 are typically required. This is computationally expensive, especially bearing in mind that there may be multiple effects at the band power level that project along the same axis in parameter space. Without a good intuition, you can find yourself applying many different offsets to the band powers with varying amplitudes and running an MCMC analysis for each case.

With a differentiable pipeline, you can investigate how band power biases project to parameter space more efficiently. Consider the following formula (https://arxiv.org/abs/1908.01626):

$$ \Delta \theta = F^{-1} \frac{\partial D_\ell}{\partial \theta} \Delta \hat{D}_\ell, $$

where $\Delta \theta$ is the shift to best-fit parameters caused by a displacement $\Delta \hat{D}_\ell$ of the band powers, $F$ is the Fisher matrix and $\partial D_\ell / \partial \theta$ the derivative of the model spectra with respect to parameters.

<span style="color:red">
<b>Exercise:</b>
</span>

Create your own version of the plot above, focussing only on the ACT data (forgetting about WMAP and Planck). Do not worry about centering your plot on the best-fit point of the ACT data, you can keep using the `fid_pars` from above - what we are interested in are the degenreacies in parameter space. Indicate the shift to the best-fit point of the ACT data when re-calibratin the $TE$ power spectrum by 100 equally-spaced factors between 0.9 and 1.1. Are you able to recover the ACT results? What advantages or disadvantages do you see in your method compared to performing multiple MCMC analyses?

Optional: come up with a different systematic (e.g. $TT$ miscalibration, contamination of the $TE$ data by an $\ell^2$ term, temperature-to-polarisation leakage, ...). How do they compare to the case of a TE recalibration for the two parameters of interest? Can you come up with any intuition for the degeneracy directions you see?


<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Discussion

In this part, you have learned how to combine `candl` with `CosmoPower-JAX` to create a differentiable pipeline from parameters to the likelihood value. You've used this to show how auto-diff can be a powerful tool for forcasting by trivialising the calculation of Fisher matrices. Moreover, you've explored how you can flexibly translate a bias at the band power level to parameters using an example from a recent paper.

---

<a id='part3'></a>

# PART III: Gradient-Powered Likelihood Exploration

We will now see how gradient information can be used to explore the likelihood surface. In particular, we will code a "traditional" and a gradient-based algorithm to find the best-fit point of the data set and compare the performance of the two algorithms.

<span style="color:blue">
<b>Background:</b>
</span>

Finding the best-fit point of the data amounts to finding the global extremum (that is not at infinity) of the likelihood function. Conventions on the sign of the likelihood differ. Some codes, such as `candl`, return the log likelihood, which is typically negative and we want to find the maximum. However, we conventionally speak about __minimisation__ when we want to find the best-fit point of the data.

In principle, the likelihood surface can take a complicated shape that can be difficult to explore. However, modern primary CMB data has become quite constraining, such that for $\Lambda \mathrm{CMB}$ we can typically approximate it as a (multivariate) Gaussian. Hence, in the algorithms below we will assume that the likelihood surface has a single peak, which we want to get to as quickly as possible.

The diagram below illustrates the idea. Imagine our likelihood compares our band powers (black points, left panel) to some model predictions (coloured lines, left panel) as we vary some cosmological parameter $\theta$. The (negative) log-likelihood is shown as a function of this parameter $\theta$ in the right panel. The best-fit point is the minimum of this function, and the goal of our minimisation algorithms will be to find it. If we only access the likelihood value, our information is limited to the function values (indicated by the coloured points in the right panel). However, if we also have gradient information we know the slope too (indicated by the coloured arrows in the right panel).

<p align="center">
  <img src="https://raw.githubusercontent.com/Lbalkenhol/candl/main/notebooks/SPT_ACT_summer_school_2024/chi2_surface.png" width="80%" alt="Differentiable Pipeline"/>
</p>

We will first write a minimisation algorithm that does not use gradient information, which we will run with CAMB and CosmoPower as theory codes. We will then develop an algorithm that uses gradient information and compare the performance of the different minmisers.


<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Traditional Minimiser

<span style="color:red">
<b>Exercise:</b>
</span>

Write an algorithm that find the best-fit point of the data. Do not use gradient-information from automatic differentiation. There are many different possible solutions. If you're unsure how to start, look at the MCMC session and modify the sampling code in there to work as a minimiser.

First, run the algorith with CAMB as a theory code and limit the maximum number of steps (i.e. CAMB evaluations) for the sake of your computer to 50. Then run the algorithm with CosmoPower as a theory code, without a hard limit to the number of steps. Does using CosmoPower help with the minimisation compared to CAMB?

The code in the two cells below will help you get started.

In [ ]:
import candl.tools

# Re-initialise CAMB - we need to grab theory spectra to higher ell than for the SPT case
CAMB_pars = camb.CAMBparams()
CAMB_pars.set_for_lmax(candl_like.ell_max+200, lens_potential_accuracy=2)
CAMB_pars_to_theory_specs = candl.interface.get_CAMB_pars_to_theory_specs_func(CAMB_pars)# this function takes a dictionary of cosmological parameters and returns a dictionary of CMB spectra

# Bundle CAMB and our likelihood together into a single function that moves from a parameter dictionary to a log like value
# Do the same for CosmoPower
pars_to_loglike_CAMB = candl.tools.get_params_to_logl_func(candl_like, CAMB_pars_to_theory_specs)
pars_to_loglike_CP = candl.tools.get_params_to_logl_func(candl_like, CP_pars_to_theory_specs)

# Evaluate the functions to make sure they run
# It's okay that these don't match exactly - the CP models were trained on CAMB spectra withs slightly higher accuracy settings
test_pars = deepcopy(fid_pars)
print(pars_to_loglike_CAMB(test_pars))
print(pars_to_loglike_CP(test_pars))

In [ ]:
# A proposal matrix for steps respecting degeneracy directions - can help you if you want to
par_order = ["H0", "ombh2", "omch2", "ns", "logA", "tau", "yp"]
proposal_cov = np.array([[ 2.25041962e+00,  1.30313866e-04, -5.37763426e-03, 1.01357518e-02, -8.31895908e-04,  5.38781094e-03, -1.68596642e-03],
                         [ 1.30313866e-04,  9.50311718e-08, -1.36037147e-07, -1.96156941e-06,  1.40972274e-06,  2.23821352e-07, 1.90509016e-07],
                         [-5.37763426e-03, -1.36037147e-07,  1.35070005e-05, -2.94150104e-05,  5.14036014e-06, -1.33453348e-05, 4.62296510e-06],
                         [ 1.01357518e-02, -1.96156941e-06, -2.94150104e-05, 2.37304854e-04, -1.47941044e-04,  2.90535404e-05, 1.74905120e-06],
                         [-8.31895908e-04,  1.40972274e-06,  5.14036014e-06, -1.47941044e-04,  9.09444696e-04,  3.80825486e-04, -9.50179538e-06],
                         [ 5.38781094e-03,  2.23821352e-07, -1.33453348e-05, 2.90535404e-05,  3.80825486e-04,  2.08304046e-04, -1.10725552e-06],
                         [-1.68596642e-03,  1.90509016e-07,  4.62296510e-06, 1.74905120e-06, -9.50179538e-06, -1.10725552e-06, 2.16146131e-05]])

# Your minimiser code

In [ ]:
# Compare the performance with CAMB and CosmoPower

<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Gradient-Based Minimiser

<span style="color:red">
<b>Exercise:</b>
</span>

Write an algorithm that find the best-fit point of the data using gradient information from auto-diff. The functions in the cell below will help you with this. Again, many different solutions are possible, but a simple yet powerful algorithm is the Newton-Raphson method (https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization). Does your gradient-based minimiser converge in less steps than the traditional one?

In [ ]:
# We grab short cuts for the derivative and the Hessian
pars_to_loglike_CP_deriv = jax.jit(jax.jacrev(pars_to_loglike_CP))
pars_to_loglike_CP_hessian = jax.jit(jax.hessian(pars_to_loglike_CP))

In [ ]:
# Your gradient-based minimiser

In [ ]:
# Compare their performance

In [ ]:
# Make a triangle plot and trace the journey of the minimisers!
# Plug your data into the variables below and you should get a nice triangle plot with the trajectories of the algorithms

# YOUR DATA GOES HERE
overall_bf_point = {} # a dictionary of the best-fit point
parameter_covariance_matrix = np.zeros((1,1)) # the parameter covariance matrix (used to draw the ellipses in the triangle plot)
parameter_covariance_matrix_order = [] # the order of parameters in the covariance matrix
traditional_minimiser_points = [{}] # a list of dictionaries - each entry is a step of the traditional minimiser
gradient_minimiser_points = [{}] # a list of dictionaries - each entry is a step of the gradient-based minimiser

# REMAINING PLOTTING CODE - DO NOT TOUCH
pars_to_plot = ["H0", "ombh2", "omch2", "ns", "logA"]# tau is prior dominated, yp is a nuisance parameters

plt.close()

# Plot parameters
sigma_plot_range = 3
sigma_levels = 2

ax = candl.plots.triangle_plot_from_cov(pars_to_plot = pars_to_plot,# which parameters to plot
                                        bf_point = overall_bf_point,# Newton best-fit point
                                        par_cov = parameter_covariance_matrix,# the parameter covariance matrix
                                        pars_in_cov = parameter_covariance_matrix_order,# the order of parameters in the covariance
                                        sigma_plot_range = sigma_plot_range,# sets the axes limits
                                        sigma_levels = sigma_levels)# how many sigma levels to plot

# Plot the Traditional minimiser
candl.plots.add_min_trajectory(ax = ax,# axis instance
                               pars_to_plot = pars_to_plot,# which parameters to plot
                               eval_points = gradient_minimiser_points,# a list of all the points in the minimiser trajectory
                               par_cov = parameter_covariance_matrix,# covariance of the best run - this is used to set the correct height in the 1d panels
                               pars_in_cov = parameter_covariance_matrix_order,# the order of parameters in the covariance
                               bf_point = overall_bf_point,# the best-fit point - again used in the 1d panels
                               base_colour = "purple")# the base colour used to show the trajectory path

# Plot the Newton minimiser
candl.plots.add_min_trajectory(ax = ax,# axis instance
                               pars_to_plot = pars_to_plot,# which parameters to plot
                               eval_points = traditional_minimiser_points,# a list of all the points in the minimiser trajectory
                               par_cov = parameter_covariance_matrix,# covariance of the best run - this is used to set the correct height in the 1d panels
                               pars_in_cov = parameter_covariance_matrix_order,# the order of parameters in the covariance
                               bf_point = overall_bf_point,# the best-fit point - again used in the 1d panels
                               dark_colours=False,
                               base_colour = "orange")# the base colour used to show the trajectory path

# Add a make-shift legend
ax[0,0].text(overall_bf_point[pars_to_plot[0]]+6*sigma_levels*np.sqrt(parameter_covariance_matrix[parameter_covariance_matrix_order.index(pars_to_plot[0]),parameter_covariance_matrix_order.index(pars_to_plot[0])]),
             0.9,
             "Gradient-Based Minimiser (NR)",
             color="orange")

ax[0,0].text(overall_bf_point[pars_to_plot[0]]+6*sigma_levels*np.sqrt(parameter_covariance_matrix[parameter_covariance_matrix_order.index(pars_to_plot[0]),parameter_covariance_matrix_order.index(pars_to_plot[0])]),
             0.7,
             "Traditional Minimiser with CosmoPower",
             color="purple")

plt.show()

<hr width="100%" style="border-style: none none dotted" color="#FFFFFF" size="6">

## Discussion

In this part, you have explored minmisation algorithms and experienced how how gradient-information can be a real gamechanger for the exploration of the likelihood surface. A simple extension you may consider (that is often done as a pre-cautionary measure in practice) is to launch multiple independent minimiser runs from random points in parameter space to ensure they all converge on the same point. Furthermore, while we have assessed in how many steps the different algorithms converge, we have not profiled (timed) them.

You can find a simple implementation of the Newton-Raphson method in the `candl.tools` module. Additionally, we can interface `candl` with `Optax` (a google-developed optimisation library for JAX, https://github.com/google-deepmind/optax) and access fanicer algorithms, such as ADAM (https://arxiv.org/abs/1412.6980). Moreover, gradient-information is also helpful for MCMC sampling - allowing you to propose more informative points to efficiently build up a representation of the posterior. If you're curious check out the `differentiable_tutorial` on the `candl` GitHub repository that develops these ideas further.

---
# Closing Remarks

We touched on many subjects in this notebook: you started handling data, accessing the data model of a likelihood, and made residual plots. You then built a differentiable pipeline and used it on real applications of CMB data analysis. Finally, you developed different minimisation algorithms - with and without gradient information - and compared their performance.

Still, this is just the tip on the ice-berg. There are many more features in `candl`. The tools library has a variety of functions that allow you to manipulate data and short-cut common analysis tasks. The interface library allows you to run `candl` likelihoods seemlessly with other cosmology codes, such as `Cobaya`, or `CLASS`. candl also includes the latest lensing likelihoods from SPT and ACT, which we haven't touched yet.

The pairing of `candl` with differentiable theory codes is particularly powerful. It trivialises the calculation of derivatives through auto-diff, which extends to Fisher matrices. This allows us to perform many calculations quickly and robustly. Moreover, for exploring the likelihood surface, gradient information is invaluable. This also extends from minimisation to MCMC sampling.

More generally, this is an exciting time for likelihood analysis in the CMB world. Many people are working on differentiable thoery codes (be they emulators, or full Boltzmann solvers), gradient-based samplers, and other techniques. This allows us to scrutinise our data more closely, which will be key as new data points with smaller and smaller error bars come in.

<p align="center">
<img src="https://media.tenor.com/enoxxJtm0yMAAAAM/neo-plugging-to-matrix.gif" width="500" alt="Welcome, Neo."/>
</p>